# Remote backend example

Enter connection a valid configuration to demo the remote backend.

In [5]:
config = {
    "ip": "82.130.102.xxx",
    "user": "karajan",
    "port": 22,
    "key": "$KARAJAN_ACCESS_DIR/id_ed25519",
}

First, import the Remote backend.

In [6]:
import os, sys, time
sys.path.append("..")

import datapro.driver.ssh

## Instantiating and connecting

Instantiate the SSH backend. The `__repr__` method is provided to display meaningful information about the state of the connection.

In [7]:
R = datapro.driver.ssh.SSH(backend=config)
print(repr(R))

<datapro.driver.ssh.SSH at 0x12166f400 (configured, inactive)>


In [8]:
R.connect()
print(repr(R))

<datapro.driver.ssh.SSH at 0x12166f400 (configured, active)>


In [9]:
from datapro.driver._backend import result_to_print_ready_dict

def print_result(result):
    _ = None
    
    if hasattr(result, "persistent"):
        _ = result.persistent
        delattr(result, "persistent")
    
    for k, v in result_to_print_ready_dict(result).items():
        print(f"# {k} {str(''):->{78-len(k)-1}}\n{v}")

    if _:
        k = "persistent"
        print(f"# {k} {str(''):->{78-len(k)-1}}")
        for k, v in _.items():
            print(f"{k+':':<10}{v!r}")
                
        result.persistent = _

## Running commands

Running commands is very simple, and can be performed via `run` or `sudo` methods, the latter having root privileges.

In [10]:
env = R.run("echo user=$HOSTNAME:$USER shell=$SHELL tty=$TTY ssh_tty=$SSH_TTY sty=$STY pwd=$PWD")

Results are instances of `invoke.runners.Result`, which wrap the following attributes:
    
- `command`: the command,
- `exited`: return code of the command,
- `stdout`: the standard output of the command,
- `stderr`: the standard error of the command.

In [11]:
print_result(env)

# command ----------------------------------------------------------------------
echo user=$HOSTNAME:$USER shell=$SHELL tty=$TTY ssh_tty=$SSH_TTY sty=$STY pwd=$PWD
# encoding ---------------------------------------------------------------------
UTF-8
# exited -----------------------------------------------------------------------
0
# stdout -----------------------------------------------------------------------
user=pc-10666:dle shell=/bin/bash tty= ssh_tty= sty= pwd=/home/dle
# stderr -----------------------------------------------------------------------



In [12]:
ps_pipe = R.run("ps ax | grep dle && >&2 echo 'Done'")
print_result(ps_pipe)

# command ----------------------------------------------------------------------
ps ax | grep dle && >&2 echo 'Done'
# encoding ---------------------------------------------------------------------
UTF-8
# exited -----------------------------------------------------------------------
0
# stdout -----------------------------------------------------------------------
25919 ?        Ss     0:00 sshd: dle [priv]
25995 ?        S      0:00 sshd: dle@notty
26000 ?        Ss     0:00 bash -c ps ax | grep dle && >&2 echo 'Done'
26002 ?        S      0:00 grep dle
# stderr -----------------------------------------------------------------------
Done


## Disconnecting

In [13]:
R.disconnect()